# Machine Learning Systems Homework 2

<a target="_blank" href="https://colab.research.google.com/github/hao-ai-lab/dsc291-PA/blob/main/pa2/mlsys_hw2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Homework due: June 2, 2024, 11:59 pm, PDT**.

Optimizing tensor programs is crucial in machine learning compilation as it ensures that operators like matrix multiplication, convolution, and reduction run efficiently on different hardware. 
In this assignment, you'll utilize a tensor program optimization interface within [TVM](https://tvm.apache.org) to schedule these common operators effectively.

* **Be prepared** -- in this homework you may need to learn many new terminologies and concepts.
* You should work on this homework **individually** -- it is not a team homework.
* This homework **requires** NVIDIA GPU with CUDA environment. You can do the homework on Google Colab (by clicking the badge above) or any other GPU server that you have access to. **Important note: we will test your submission on Google Colab. So if you do the homework on other environments, we recommend you to test it on Colab before submission.**
* This homework is pure Python. No C++ is needed in this homework.
* Please check out the end of this notebook for the homework submission requirement.
* Please do not share your solution on publicly available websites (e.g., GitHub).
* **About testing and grading.** You can submit your homework for multiple times. Your submission (only your last one) will be read and graded **manually** after the homework deadline. **This means you will not be able to see your scores immediately after submission.** The content of this notebook below describes some possible ways for you to check and test by yourself.

## Preparation

### Using Google Colab

If you are using Google Colab environment, please make a copy of this notebook file by selecting "Save a copy in Drive" from the "File" menu, and then run the code block below to set up workspace. After cloning, you will see the cloned repository in the "Files" bar on the left.

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p dsc291
%cd /content/drive/MyDrive/dsc291
!git clone https://github.com/hao-ai-lab/dsc291-PA.git
%cd /content/drive/MyDrive/dsc291/dsc291-PA/pa2

Then run the following cell to install TVM dependency.

**Note.** The PyPI package installed below only has CPU support.
It is fine to use this CPU package when you implement the five tasks of part 1.
We will have instructions on how to install the GPU package
later on.

In [ ]:
!python3 -m pip install mlc-ai -f https://mlc.ai/wheels

We can validate that the dependency is successfully installed by running the following.

In [ ]:
import tvm
tvm.__path__

['/usr/local/lib/python3.10/dist-packages/tvm']


### Using local environment

If you are using local/server environment (requiring NVIDIA GPU and CUDA ≥ 11.7), please clone this repository.

```shell
git clone https://github.com/hao-ai-lab/dsc291-PA.git
cd dsc291-PA/pa2
export PYTHONPATH=.:$PYTHONPATH
```

We recommend you to create a conda environment and install the dependency package in the environment.
```shell
conda create --name dsc291 python=3.11
conda activate dsc291
# Replace "cu118" with your local CUDA version (e.g., cu121, cu122, etc.)
python3 -m pip install --force-reinstall mlc-ai-cu118 -f https://mlc.ai/wheels
```

Please also select the Python interpreter in this conda environment and use it to run this notebook.
We can validate that the dependency is successfully installed by running the following.

In [ ]:
import tvm
tvm.__path__

['/path/to/conda/envs/15442/lib/python3.11/site-packages/tvm']


## Part 1. Optimize Matrix Multiplication (80 pt)

In this part, we will walk you through the process of manually optimize and fuse the ""GeMM + ReLU + add" operator with TVM
and run it on the NVIDIA GPU in your environment.

The "GeMM + ReLU + add" pattern is common in machine learning models when activation
and residual-add operations are involved.
Formally, for given 2-dim tensors $A$, $B$, and $C$, we want to compute the following $D$:

$$
D = \mathrm{ReLU} (A B) + C.
$$

We define this operator using [TensorIR](https://arxiv.org/pdf/2207.04296.pdf),
the tensor program level intermediate representation (IR) in TVM, as follows.
You can also find this definition in `gemm_relu_add.py`.

```python
from tvm import tir
from tvm.script import tir as T

M = 2048
N = 2048
K = 2048


@T.prim_func
def gemm_relu_add(
    A: T.Buffer((M, K), "float32"),
    B: T.Buffer((K, N), "float32"),
    C: T.Buffer((M, N), "float32"),
    D: T.Buffer((M, N), "float32"),
) -> None:
    matmul = T.alloc_buffer((M, N), "float32", scope="global")
    relu = T.alloc_buffer((M, N), "float32", scope="global")
    # Compute GeMM
    for i, j, k in T.grid(M, N, K):
        with T.block("gemm"):
            vi = T.axis.spatial(M, i)
            vj = T.axis.spatial(N, j)
            vk = T.axis.reduce(K, k)
            with T.init():
                matmul[vi, vj] = T.float32(0)
            matmul[vi, vj] += A[vi, vk] * B[vk, vj]
    # Compute ReLU
    for i, j in T.grid(M, N):
        with T.block("relu"):
            vi = T.axis.spatial(M, i)
            vj = T.axis.spatial(N, j)
            relu[vi, vj] = T.max(matmul[vi, vj], T.float32(0))
    # Compute add
    for i, j in T.grid(M, N):
        with T.block("add"):
            vi = T.axis.spatial(M, i)
            vj = T.axis.spatial(N, j)
            D[vi, vj] = relu[vi, vj] + C[vi, vj]


sch = tir.Schedule(gemm_relu_add)
```

Let's go through the above code and look into its components.

* The function is designed to take four tensors `A`, `B`, `C` and `D` as inputs, each with thier corresponding share and dtype.
Notably, D is passed in as an argument rather than being returned by the function. This approach, known as "destination-passing style," 
involves pre-allocating the destination tensor before calling the function and then passing it in as an argument.
This style is commonly used in low-level operator libraries like [cuBLAS](https://docs.nvidia.com/cuda/cublas/index.html) and in tensor program abstractions within machine learning compilers.
* The initial two lines within the function body establish tensors for storing intermediate results from the GeMM and ReLU operations. 
At this stage, their memory scope is global.
* Next, there are the three sequential for-loop blocks defining the computation.
  * The first one defines the GeMM computation. For a given `vi` and `vj`, the loop body
  reads from `A` and `B`, sums up the values along `vk`, and stores the summation result into
  the intermediate tensor `matmul`, with `T.float32(0)` being the initial value of the summation.
  * The second one defines the ReLU computation, which element-wisely takes the maximum
  of `matmul` and zero.
  * The third one describes the element-wise addition of the ReLU result and the input tensor `C`.
* After defining the function, we create a `tir.Schedule` instance with regard to this function.
`tir.Schedule` is the core tool in this homework you will use to optimize this function.
`tir.Schedule` provides the a set of function transformations (which we call "schedule primitives")
that could accelerate the function execution on hardware.
We will introduce these schedule primitives later.

Your goal in this part is to optimize this GeMM + ReLU + add workload so that it can efficiently run on GPU.
Usually, optimizing such a workload of matrix multiplication with epilogues (e.g., ReLU and add here)
for GPU, five major steps are required:

1. shared memory tiling,
2. register tiling,
3. cooperative fetching,
4. write cache.
5. epilogue fusion.

In grading your submission, we will access your optimization from two perpectives:
* Firstly, we will ensure numerical correctness remains intact post-optimization. This means that 
your optimized codes should yield the same numerical results as the original naive implementation.
We have provided the tests for you to ocheck the numerical correctness of your optimization.
But you may not be able to run the tests before you have implemented all optimizations.
* Secondly, we will evaluate whether you have correctly implemented the optimizations outlined above.
While there is no automated test for this, we will manually review your submission to ensure each task
is properly addressed. Additionally, we have provided a reference implementation (`reference.py`) as your
guidance to complete all five tasks, **though your final function need not match it exactly**.



Now, let's get started.

### Task 1. Shared memory tiling (20 pt)

Our optimization primarily focuses on enhancing the GeMM (General Matrix Multiply) component. 
GeMM involves the most intensive computation and memory accesses among the three stages (GeMM, ReLU, and addition) in this workload.
By optimizing GeMM, subsequent optimizations for the remaining parts can be tailored based on the improvements made to GeMM.

You have learned about the concept of **thread blocks** in GPU architectures from the lectures.
To optimize a GeMM workload, people usually partition the destination matrix (`matmul` for our case)
into multiple tiles along both the row and column dimensions, and assign one thread block
for the computation of each tile.

For example, in the figure below, a thread block computes a `(tile_x, tile_y)` tile.
That is to say, this particular thread block will iterate over a `(tile_x, K)` region of `A`
and a `(K, tile_y)` region of `B`, compute the matrix multiplication of these two regions,
and write the results into the `(tile_x, tile_y)` tile.
We can use Python notation to formally describe the tiling:
for a given thread block, assuming the tile that this thread block computes
starts from `offset_x` on the `M` dimension and `offset_y` on the `N` dimension,
then the thread block computes the following:
```python
matmul[offset_x : offset_x + tile_x, offset_y : offset_y + tile_y] = (
    A[offset_x : offset_x + tile_x, :] @ B[:, offset_y : offset_y + tile_y]
)
```

<img src="figure/shared-memory-tiling.jpg" alt="figure/shared-memory-tiling.jpg" width="100%">

The next step is about reducing the memory access pressure of the GeMM operator.
This is done via leveraging **shared memory** available in GPU, which you also learned about from lectures:
rather than directly reading from `A` and `B` (which reside in global memory)
at the time of computing multiplication, each thread block first loads the `A` region
and `B` region it needs from global memory to shared memory.

Just like CPU cache, while shared memory can offer much faster access speed,
it has relatively smaller size than global memory.
Usually, the size of shared memory available in a thread block is no more than 48KB.
In our case, given each float32 element has four bytes,
the `A` region and `B` region that a thread block needs
have size `(tile_x * K * 4) + (tile_y * K * 4)` byte,
which is far more than the available shared memory size a thread block can use.
To address this issue, you also need split the long stripe of `A` and `B` into
multiple chunks along the `K` dimension.
By doing this, now you are able to go through `A` chunks and `B` chunks one at a time simultaneously,
and load a single `A` tile and `B` chunk into shared memory, which fits the available
shared memory size.
This can be formally written as

```python
matmul[offset_x : offset_x + tile_x, offset_y : offset_y + tile_y] = sum(
    A[offset_x : offset_x + tile_x, k : k + tile_k]
    @ B[k : k + tile_k, offset_y : offset_y + tile_y]
    for k in range(0, n, tile_k)
)
```


We can analyze how much global memory accesses we reduce by leveraging shared memory:

* There are $M \times K$ elements in $A$ and $N \times K$ elements in $B$.
When not using shared memory, each element of $A$ is read for $N$ times,
and each element of $B$ is read for $M$ times.
So number of accesses to $A$ and $B$ in global memory are both $MNK$.
* With shared memory tiling, the number of times each element of $A$ is read becomes $\frac{N}{\mathrm{tile}_y}$,
and the number of times each element of $B$ is read becomes $\frac{M}{\mathrm{tile}_x}$.
And the total number of accesses to $A$ and $B$ are divided by $\mathrm{tile}_y$ and $\mathrm{tile}_x$ respectively.


You need to implement the shared memory tiling in the `shared_memory_tiling` function in `gemm_relu_add.py`.
Below are the schedule primitives you will use in this task.
We provide [another Jupyter notebook](schedule_example.ipynb) that walks you through an example
of using some of these schedule primitives for your reference.
Meanwhile, you can click the links below to check out their documentations,
which contain a basic example on how to use the primitive.
We also provide instructions in your TODO area in `gemm_relu_add.py`.

- `split` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.split)
  - It splits one loop into multiple loops that collectively iterate the iteration space of the original loop.
  We use `split` for tiling.
- `reorder` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.reorder)
  - It reorders the specified contiguous loops into a new order.
  We apply `reorder` for loops after splitting for tiling.
- `bind` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.bind)
  - It binds a loop to `blockIdx.x/y/z` or `threadIdx.x/y/z` on GPU,
  so that we can specify the area to compute of a thread block or a single thread.
- `cache_read` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.cache_read)
  - It generates a cache stage for the specified region in the specified memory scope.
  We use `cache_read` to generate the read stages from global memory to shared memory.
- `compute_at` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.compute_at)
  - It moves a computation (e.g., the shared memory read stage) to the location under the specified loop.
  We use `compute_at` to move the shared memory read stages to the right location.

You can run `gemm_relu_add.py` to see how the function looks like after your optimization.
Note: you can insert `sch.show()` at any position to print out the function,
so that you can visually see how the function looks like.

In [ ]:
!python3 gemm_relu_add.py

### Task 2. Register tiling (20 pt)

In the previous subsection, we focus on accelerating the `matmul` region that **a thread block collectively computes**.
In this subsection, we dive deeper to figure out the `matmul` region that **a single thread computes**.

The idea is pretty much similar to the shared memory tiling.
We can also let a single thread compute a tile in `matmul`.
For example, we assign each thread in a thread block to compute a tile of size `(thread_tile_x, thread_tile_y)`.
And moreover, rather than directly load `A` and `B` from shared memory when computing the tile,
we again first load `A` and `B` from shared memory to **local registers**.
Registers are local memory in a running CUDA thread that is running,
and can be accessed in much shorter time than shared memory.
Similarly, we also need to do thread-level tiling along the `K` dimension,
because each thread has very limited number of available registers.
And our goal is to make sure the data that we want to load from `A` and `B` in each thread-level tile can fit into the registers.
With register tiling, we use $(\mathrm{thread\_tile}_x + \mathrm{thread\_tile}_y) \times \mathrm{thread\_tile}_k$
registers for reading data from the shared memory of `A` and `B` in total.


<img src="figure/register-tiling.jpg" alt="figure/register-tiling.jpg" width="50%">



It worths noting that, by fixing the thread tile size, we equivalently fixed the number
of threads in each thread block:
a thread block computes a tile of `(tile_x, tile_y)`, and a single thread computes a tile of
`(thread_tile_x, thread_tile_y)`.
So in a thread block, `blockDim.x` is `tile_x / thread_tile_x`, `blockDim.y` is `tile_y / thread_tile_y`,
and the total number of threads in a thread block is `(tile_x / thread_tile_x) * (tile_y / thread_tile_y)`.


You need to implement the register tiling in the `register_tiling` function in `gemm_relu_add.py`.
As mentioned above, your implementation of register tiling should follow the same pattern and steps
as your implementation of shared memory tiling.
So make sure you understand each step of the shared memory tiling before starting this task.
The schedule primitives you will use in this task is the same as task 1.
And please note that for register tiling, you should use `"local"` (which means local registers)
as the scope of `cache_read`, rather than `"shared"` which you used in task 1.

In [ ]:
!python3 gemm_relu_add.py

### Task 3. Cooperative fetching (20 pt)

In both shared memory tiling and register tiling, we've designated a `matmul` region 
for each thread to compute, completing our setup for the GeMM computation. However, 
there are still a few optimization steps remaining. The first one is **cooperative fetching**, 
focusing on optimizing the memory load from global memory to shared memory for both matrices `A` and `B`.

In the "shared memory tiling" subsection, we established stages for a thread block 
to fetch data from matrices `A` and `B` into their respective shared memory. 
However, the specific manner in which the data is fetched remains unspecified. 
Particularly, we haven't specified which elements of `A`/`B` each individual thread 
should read. Without this specification, each thread within a thread block will, 
by default, load the entire `A`/`B` regions assigned to the thread block into the shared memory. 
As shared memory is accessible to all threads within the block, this default behavior 
results in redundant global memory reads and shared memory writes, 
failing to effectively reduce the number of global memory accesses.

This motivates us to implement cooperative fetching.
Cooperative fetching means all the threads in a thread block
collectively ("*cooperative*") load values from global memory to shared memory ("*fetching*").
To make sure each element in the tile is read only once (by some thread),
we can divide the number of elements in the shared memory tile evenly by
the number of threads, yielding the number of elements each thread loads.
Then, we let all the threads in the thread block to load values in memory order.

For our example, we can calculate that each thread in a thread block needs to
load `thread_tile_x * thread_tile_y * tile_k / tile_y` elements from `A`,
and likewise, `thread_tile_x * thread_tile_y * tile_k / tile_x` elements from `B`.
If we denote the number of total threads (`(tile_x / thread_tile_x) * (tile_y / thread_tile_y)`) as
`num_threads`, then when loading an `A` tile to shared memory, in the first round all the threads
load the first continuous `num_threads` elements, in the second round they load
the next continuous `num_threads` elements, and keep loading for
`thread_tile_x * thread_tile_y * tile_k / tile_y` times.
This applies to the shared memory load of `B` in the same way.

[The other Jupyter notebook](schedule_example.ipynb) contains a simple example of
cooperative fetching that you might find helpful for understanding,
if you have not went through it.

You need to implement the cooperative fetching in the `cooperative_fetching` function in `gemm_relu_add.py`.
As you can see in the code, since the cooperative fetching for `A` and `B` are the same,
we can write a common schedule function and apply it to the shared memory read stages
of both `A` and `B`.
The new schedule primitive you will use in this task is
`fuse` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.fuse),
which, on the contrary to `split`, fuses multiple loops into a single loop.

In [ ]:
!python3 gemm_relu_add.py

### Task 4. Write Cache (10 pt)

Now, we proceed to the stage of writing the GeMM computation results into `matmul`. 
Up to this point, our GeMM computation still directly writes the results into the global 
memory of `matmul`, with each location of `matmul` being written to `K` times due to 
the summation over the `K` dimension. Clearly, this approach is suboptimal.

Instead of directly writing the computation results into global memory, 
we can accumulate the summation in local registers. Subsequently, we can write 
the final result into global memory fewer times after the accumulation process is completed.


You need to implement the write cache in the `write_cache` function in `gemm_relu_add.py`.
In this task, you will use the schedule primitive
`cache_write` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.cache_write)
and
`reverse_compute_at` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.reverse_compute_at)
to put the write cache stage at the right location.
Similar to `cache_read` which generates a read cache stage for a computation,
`cache_write` generates a write cache stage for a computation at the specified scope.
`reverse_compute_at` works in basically the same way as `compute_at`,
with the difference being `compute_at` moves a computation **in front of** the target loop into the target loop,
while `reverse_compute_at` moves a computation **behind** the target loop into the target loop.

In [ ]:
!python3 gemm_relu_add.py

### Task 5. Epilogue Fusion (10 pt)

The preceding subsections have covered common techniques utilized to optimize a 
GeMM workload. Now, we shift our focus to the ReLU and addition operators. 
While these operators are computationally lighter compared to GeMM, they still 
consume a considerable amount of time due to their direct read/write operations 
from/to global memory. This inefficiency can be addressed by incorporating the 
lightweight epilogue operators (ReLU and addition) directly into the write-back 
stage of the scheduled GeMM workload. During this stage, the results of GeMM are 
still stored in registers, optimizing the elimination of redundant memory accesses.




You need to implement the epilogue fusion in the `epilogue_fusion` function in `gemm_relu_add.py`,
where you will use `get_block` to retrieve the computation of addition and ReLU,
and use `reverse_compute_inline` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir.html#tvm.tir.Schedule.reverse_compute_inline)
to inline the addition and ReLU into the write-back stage of GeMM.

In [ ]:
!python3 gemm_relu_add.py

---

### Task 6. Evaluating your optimization (10 pt)

Now that you have completed all five steps of the optimization. Please compare 
the execution time of your optimized implementation and that of the naive implementation by following the instructions below and report the time in the 
file [time_difference.txt](time_difference.txt).

Please also try to tune the tile size in your optimized implementation, report how the time changed and came up with some possible reasons in the file [time_difference.txt](time_difference.txt).

#### If you are using Google Colab...

Now we need to switch to GPU runtime and install the PyPI package with CUDA support before
running tests.
Please go to "Runtime - Change runtime type" in the navigation bar, and select "T4 GPU".
Then run the cells below to install the PyPI package, verify the installation,
and mount the Google Drive again.

**Note 1.** Be sure to change runtime type to use T4 GPU.
Or otherwise you will see library error when importing TVM in Python below.

**Note 2.** Please disconnect the GPU runtime when you are not using it.
Otherwise you may hit the Colab GPU usage limit and will be temporarily not able to
connect to a GPU in Colab if you are a free Colab user.

In [ ]:
!python3 -m pip install mlc-ai-cu122 -f https://mlc.ai/wheels

In [ ]:
import tvm
tvm.__path__

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/dsc291/dsc291-PA/pa2

---

Now that you have finished implementing all the optimizations,
you can test the numerical correctness of your code via running the following.

In [ ]:
!python3 evaluate.py --test

Passing test round 0...
Passing test round 1...
Passing test round 2...
Passing test round 3...
Passing test round 4...
Passed all tests.


You can evaluate the execution time of function after your optimization via running the following:

In [ ]:
!python3 evaluate.py --evaluate-manual

To help you get a sense of how much performance your optimization gains,
we can evaluate a naive GeMM + ReLU + add function with little optimizations.
Ideally, you should find that your optimization is times faster than the naive optimization.

In [ ]:
!python3 evaluate.py --evaluate-naive

You can print out the CUDA source code of your scheduled workload that TVM generates,
and compare it with the Python script you printed
via `sch.show()`.
As a practice, try to point out which parts in the CUDA source code
are the shared memory, local registers,
the main GeMM computation, cooperative fetching, write cache,
and the fused epilogue respectively.

In [ ]:
!python3 evaluate.py --show-cuda

## Part 2. Homework Feedback (0 pt)

This is the first time we offer this course, and we appreciate any homework feedback from you.
You can leave your feedback (if any) in `feedback.txt`, and submit it together with the source code.
Possible choices can be:

- Are the notebooks and instructions clear enough?
- How difficult do you think this homework is?
- How much time does the homework take? Which task takes the most time?
- Which part of the homework do you feel hard to understand?
- And any other things you would like to share.

Your feedback will be very useful in helping us improve the homework quality
for next years.


## How to Submit Your Homework

In the home directory for the assignment, execute the command

In [ ]:
!make handin.tar

This will create an archive file with `gemm_relu_add.py`, `time_difference.txt` and `feedback.txt`.
You can check the contents of `handin.tar` to make sure it contains all the needed files:

In [ ]:
!tar -tf handin.tar

It is expected to list the three files:
```
gemm_relu_add.py
time_difference.txt
feedback.txt
```

Then, please go to Gradescope and submit the file `handin.tar`.

This assignment is not automatically graded, and you will not receive immediate feedback upon submission.
You can submit multiple times, but only your final submission will be graded.